# Aquifer Storage and Recovery with buoyancy in a multi-layer system with MODFLOW6 and Flopy

In this notebook, we will learn how to:
1. Simulate both injection and recovery of freshwater with a well (radial flow).
2. Include the effect of buoyancy
3. Perform two cycles

In [ ]:
# import the necessary packages
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (5, 3) # set default figure size
import flopy as fp  # import flopy and call it fp

## Description of the flow problem
Consider radial flow in a confined aquifer. The aquifer extends in the $r$ direction from $r=0$ to $r=R$, where $R$ is chosen far enough away not to effect the solution.  Water is injected by the well at a rate $Q$. The head is fixed at $r=R$ to $h_R$. Flow is considered to be at steady state instantaneously. 

The initial salt concentration is equal to $c_s$ everywhere. Injection of fresh water with concentration $c_f$ starts at $t=0$ and last for $t_\text{in}$ days, after which water is extracted at the same rate $Q$ for $t_\text{out}$ days.

The density $\rho$ of the water is approximated using the following linear relation:
\begin{equation}
\rho = \frac{\text{d}\rho}{\text{d}c}(c-c_\text{ref}) + \rho_\text{ref}
\end{equation}
where the reference concentration is $c_\text{ref}=0$ and the reference density is $\rho_\text{ref}=1000$ kg/m$^3$ (i.e., freshwater) and $\frac{\text{d}\rho}{\text{d}c}$ is the specified gradient such that $c=35$ kg/m$^3$ results in a density of 1025 kg/m$^3$.  



In [ ]:
# domain size and boundary conditions
rw = 0.2 # radius of well, m
R = 80 # length of domain, m
hR = 0 # head at right side of domain

# aquifer parameters
k = 20 # hydraulic conductivity, m/d
kv = k / 10 # vertical hydraulic conductivity, m/d
H = 20 # aquifer thickness, m
npor = 0.35 # porosity, -

# flow
Q = 500 # injection and extraction rate, m^3/d

# transport
alphaL = 0.5 # longitudinal dispersivity in horizontal direction
alphaT = alphaL / 10
diffusion_coef = 0

# concentration
cs = 35 # initial concentration, g/L (=kg/m^3)
cf = 0 # concentration injected water, g/L

# buoyancy
rhoref = 1000 # reference density, kg/m^3
cref = 0 # reference concentration, kg/m^3
drhodc = 0.7143  # Slope of the density-concentration line

# space discretization
delr = 0.5 # length of cell along row (in x-direction), m
delc = 1 # width of cells normal to plane of flow (in y-direction), m
nlay = 40 # number of layers (was 10)
nrow = 1 # number of rows
ncol = round((R - rw) / delr) # number of columns
z = np.linspace(0, -H, nlay + 1) # top and bottom(s) of layers
zc = 0.5 * (z[:-1] + z[1:]) # center of cells, used for contouring
r = rw + np.cumsum(delr * np.ones(ncol)) - 0.5 * delr # rightside of cell minus half the cell length
rb = np.hstack((rw, rw + np.cumsum(delr * np.ones(ncol)))) # radial coordinates of boundaries of cells, m

# radialize parameters:
theta = 2 * np.pi
krad = k * r * theta * np.ones((nlay, nrow, ncol))
kvertrad = kv * r * theta * np.ones((nlay, nrow, ncol))
nporrad = npor * r * theta * np.ones((nlay, nrow, ncol))

# time discretization
tin = 50 # injection time, d
delt = 0.5 # time step, d
nstepin = round(tin / delt) # computed number of steps during injection, integer
tout = 50 # extraction time, d
delt = 0.5 # time step, d
nstepout = round(tout / delt) # computed number of steps during extraction, integer

# model name and workspace
modelname = 'modelbuoy' # name of model
gwfname = modelname + 'f' # name of flow model
gwtname = modelname + 't' # name of transport model
modelws = './' + modelname # model workspace to be used

## Create Simulation
Changes in the simulation: None.

In [ ]:
# simulation
sim = fp.mf6.MFSimulation(sim_name=modelname, # name of simulation
                          version='mf6', # version of MODFLOW
                          exe_name='/Users/mark/bin/mf6', # absolute path to MODFLOW executable
                          sim_ws=modelws, # path to workspace where all files are stored
                         )

# time discretization
tdis = fp.mf6.ModflowTdis(simulation=sim, # add to the simulation called sim (defined above)
                          time_units="DAYS", 
                          nper=2, # number of stress periods 
                          perioddata=[[tin, nstepin, 1],
                                      [tout, nstepout, 1]], # period length, number of steps, timestep multiplier
                         )

## Create groundwater flow model (`gwf`)
Changes in the groundwater flow model:
* Use the converted value of $k$ in the npf package and use the logarithmic averaging option to compute cell-by-cell hydraulic conductivities.
* Add the (converted) vertical hydraulic conductivity
* Use the discharge $Q$ in the well package.
* Buoyancy is specified with the buoyancy package

In [ ]:
# groundwater flow model
gwf = fp.mf6.ModflowGwf(simulation=sim, # add to simulation called sim
                        modelname=gwfname, # name of gwf model
                        save_flows=True, # make sure all flows are stored in binary output file
                       )

# iterative model solver
gwf_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation called sim
                             filename=gwf.name + '.ims', # file name to store ims
                             linear_acceleration="BICGSTAB", # use BIConjuGantGradientSTABalized method
                             inner_dvclose=1e-6, # iterate a bit longer for convergence
                            )                                                                                                
# register solver
sim.register_ims_package(solution_file=gwf_ims, # name of iterative model solver instance
                         model_list=[gwf.name], # list with name of groundwater flow model
                        )   

# discretization
gwf_dis = fp.mf6.ModflowGwfdis(model=gwf, # add to groundwater flow model called gwf
                               nlay=nlay, 
                               nrow=nrow, 
                               ncol=ncol, 
                               delr=delr, 
                               delc=delc, 
                               top=z[0], 
                               botm=z[1:], 
                              )

# aquifer properties
gwf_npf  = fp.mf6.ModflowGwfnpf(model=gwf, 
                                k=krad, # horizontal k value
                                k33=kvertrad, # vertical k value
                                save_flows=True, # save the flow for all cells
                               )
    
# initial condition
gwf_ic = fp.mf6.ModflowGwfic(model=gwf, 
                             strt=hR, # initial head used for iterative solution
                            )

# wells
wellin = []
wellout = []
for ilay in range(nlay):
    wellin.append([(ilay, 0, 0),  Q / nlay, cf])  # [(layer, row, col), U, concentration]
    wellout.append([(ilay, 0, 0),  -Q / nlay, cf]) # specified concentration is not used, but must be specified 
wel_spd = {0: wellin, 1: wellout} # stress period data for periods 0 and 1
gwf_wel = fp.mf6.ModflowGwfwel(model=gwf, 
                               stress_period_data=wel_spd, 
                               auxiliary=['CONCENTRATION'],
                               pname='WEL1', # package name
                              )

# constant head 
chd = []
for ilay in range(nlay):
    chd.append([(ilay,  0,  ncol-1), hR, cs]) # [(layer, row, col), head, concentration]
chd_spd  = {0: chd, 1: chd}    # Stress period data
gwf_chd = fp.mf6.ModflowGwfchd(model=gwf, 
                               stress_period_data=chd_spd, 
                               auxiliary=['CONCENTRATION'],
                               pname='CHD1', # package name
                              )

# buoyancy
buy = fp.mf6.ModflowGwfbuy(model=gwf,
                           packagedata=[0, drhodc, cref, gwtname, 'CONCENTRATION'],
                           denseref=rhoref, # reference concentration
                           nrhospecies=1, # number of species
                           density_filerecord=f"{gwf.name}.dst", # file name
                           pname='BUY1', 
                          )
    
# output control
oc = fp.mf6.ModflowGwfoc(model=gwf, 
                         saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")], # what to save
                         budget_filerecord=f"{gwfname}.cbc", # file name where all budget output is stored
                         head_filerecord=f"{gwfname}.hds", # file name where all head output is stored
                        )

## Create groundwater transport model (`gwt`)
Changes in the transport model: None.

In [ ]:
# groundwater transport model
gwt = fp.mf6.ModflowGwt(simulation=sim, 
                        modelname=gwtname, # name of groundwater transport model
                       )

# iterative model solver
gwt_ims  = fp.mf6.ModflowIms(simulation=sim,
                             filename=gwt.name + '.ims', # must be different than file name of gwf model ims
                             linear_acceleration="BICGSTAB",
                             inner_dvclose=1e-6,
                            ) 
sim.register_ims_package(solution_file=gwt_ims, 
                         model_list=[gwt.name],
                        )

# discretization
gwt_dis = fp.mf6.ModflowGwtdis(model=gwt, 
                               nlay=nlay, 
                               nrow=nrow, 
                               ncol=ncol, 
                               delr=delr, 
                               delc=delc, 
                               top=z[0], 
                               botm=z[1:], 
                              )

# mobile storage and transfer
gwt_sto = fp.mf6.ModflowGwtmst(model=gwt, 
                               porosity=nporrad, # porosity
                               save_flows=True,
                              )

# initial condition
gwt_ic = fp.mf6.ModflowGwtic(model=gwt, 
                             strt=cs, # initial concentration
                            ) 

# source sink mixing
sourcelist = [("WEL1", "AUX", "CONCENTRATION"), ("CHD1", "AUX", "CONCENTRATION")]
ssm = fp.mf6.ModflowGwtssm(model=gwt, 
                           sources=sourcelist, 
                           save_flows=True,
                           pname='SSM1', 
                          )

# advection
adv = fp.mf6.ModflowGwtadv(model=gwt,  
                           scheme="TVD",  # use Total Variation Diminishing (TVD)
                           pname='ADV1',
                          )

# dispersion
dsp = fp.mf6.ModflowGwtdsp(model=gwt, 
                           alh=alphaL,
                           ath1=alphaT, 
                           diffc=diffusion_coef,
                           pname='DSP1', 
                          )

# output control
oc = fp.mf6.ModflowGwtoc(model=gwt,
                         saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")], # what to save
                         budget_filerecord=f"{gwtname}.cbc", # file name where all budget output is stored
                         concentration_filerecord=f"{gwtname}.ucn", # file name where all concentration output is stored
                        )

Changes in interaction: None

In [ ]:
fp.mf6.ModflowGwfgwt(simulation=sim, 
                     exgtype="GWF6-GWT6", 
                     exgmnamea=gwf.name , 
                     exgmnameb=gwt.name , 
                     filename=f"{modelname}.gwfgwt",
                    );

## Write input files and solve model

In [ ]:
sim.write_simulation(silent=True)
success, _ = sim.run_simulation(silent=True) 
if success == 1:
    print('Model solved successfully')
else:
    print('Solve failed')

## Read concentration data and make contour plot of the concentration

In [ ]:
cobj = gwt.output.concentration() # get handle to binary concentration file
c = cobj.get_alldata().squeeze() # get the concentration data from the file
times = np.array(cobj.get_times()) # get the times and convert to array

In [ ]:
plt.subplot(111, xlim=(0, 80), ylim=(-20, 0), xlabel='x (m)', ylabel='z (m)', aspect=1)
cset = plt.contour(r, zc, c[40], np.arange(5, 35, 5), cmap='coolwarm')
plt.colorbar(cset, shrink=0.35, aspect=4);

In [ ]:
from ipywidgets import interact

#def contour(tstep):
#    plt.subplot(111, xlim=(0, 50), ylim=(-20, 0), xlabel='r (m)', ylabel='z (m)')
#    plt.contour(r, zc, c[tstep], np.arange(5, 35, 5), cmap='coolwarm')

def contour(t=delt):
    tstep = round(t / delt) - 1
    plt.subplot(111, xlim=(0, 80), ylim=(-20, 0), xlabel='x (m)', ylabel='z (m)', aspect=1)
    cset = plt.contour(r, zc, c[tstep], np.arange(5, 35, 5), cmap='coolwarm')
    plt.colorbar(cset, shrink=0.35, aspect=4)

interact(contour, t=(delt, tin + tout, delt));


In [ ]:
delt, tin + tout, delt

## Homework

### Problem 3
Compute and report the mass balance during the last time step of extraction

### Problem 4
Compute and report the recovery efficiency of cycle 1 and the recovery efficiency of cycle 2 using $c_\text{limit}=1$ g/L. Note that $c_\text{limit}$ is the limit of the average concentration of the water pumped by the well. 

### Problem 5
Compute and report the recovery efficiency of cycle 1 and the recovery efficiency of cycle 2 using $c_\text{limit}=1$ g/L and 40 layers of equal thickness rather than 10 layers.

## Answers

### Answer to Problem 3

In [ ]:
# during injection (not asked)
itime = 1
delM1ext = (cf - cs) * Q * delt
print('delM1: ', delM1ext)
delconc = (c[itime, :, :] - c[itime - 1, :, :])
area = (rb[1:] ** 2 - rb[:-1] ** 2) * np.pi
delM2 = np.sum(delconc * area) * H / nlay * npor
print('delM2: ', delM2)

In [ ]:
itime = 199 # last time step
delM1ext = (-np.mean(c[itime, :, 0]) + cs) * Q * delt
print('delM1: ', delM1ext)
delconc = (c[itime, :, :] - c[itime - 1, :, :])
area = (rb[1:] ** 2 - rb[:-1] ** 2) * np.pi
delM2 = np.sum(delconc * area) * H / nlay * npor
print('delM2: ', delM2)

In [ ]:
# during injection (not asked)
itime = 1
delM1ext = (cf - cs) * Q * delt
print('delM1: ', delM1ext)
delconc = (c[itime, :, :] - c[itime - 1, :, :])
area = (rb[1:] ** 2 - rb[:-1] ** 2) * np.pi
delM2 = np.sum(delconc * area) * H / nlay * npor
print('delM2: ', delM2)

In [ ]:
itime = 599 # last time step
delM1ext = (-np.mean(c[itime, :, 0]) + cs) * Q * delt
print('delM1: ', delM1ext)
delconc = (c[itime, :, :] - c[itime - 1, :, :])
area = (rb[1:] ** 2 - rb[:-1] ** 2) * np.pi
delM2 = np.sum(delconc * area) * H / nlay * npor
print('delM2: ', delM2)

### Answer to Problem 4

In [ ]:
# initial condition
gwt_ic = fp.mf6.ModflowGwtic(model=gwt, 
                             strt=cs, # initial concentration
                            ) 

sim.write_simulation(silent=True)
success, _ = sim.run_simulation(silent=True) 
if success == 1:
    print('Model solved successfully')
else:
    print('Solve failed')

cobj = gwt.output.concentration() # get handle to binary concentration file
c = cobj.get_alldata().squeeze() # get the concentration data from the file

climit = 1 # g/L
for itime in range(300, 600):
    if np.mean(c[itime, :, 0]) > climit:
        itime0 = itime
        break
rec_eff = ((times[itime - 1] - tin) * Q) / (tin * Q) # U not needed as injection and extraction rates are the same
print(f'recovery efficiency = {rec_eff * 100:.1f} %')

# initial condition
gwt_ic = fp.mf6.ModflowGwtic(model=gwt, 
                             strt=c[itime - 1], # initial concentration
                            ) 

sim.write_simulation(silent=True)
success, _ = sim.run_simulation(silent=True) 
if success == 1:
    print('Model solved successfully')
else:
    print('Solve failed')

cobj = gwt.output.concentration() # get handle to binary concentration file
c = cobj.get_alldata().squeeze() # get the concentration data from the file

climit = 1 # g/L
for itime in range(300, 600):
    if np.mean(c[itime, :, 0]) > climit:
        itime0 = itime
        break
rec_eff = ((times[itime - 1] - tin) * Q) / (tin * Q) # U not needed as injection and extraction rates are the same
print(f'recovery efficiency = {rec_eff * 100:.1f} %')